In [63]:
import sqlalchemy
import imp
import pandas as pd
import os
import preprocessing

f, filename, desc = imp.find_module('db_helpers', ['../data'])
try:
    db_helpers = imp.load_module('db_helpers', f, filename, desc)
finally:
    f.close()
connect = db_helpers.connect

[autoreload of db_helpers failed: Traceback (most recent call last):
  File "/home/tyrone/anaconda2/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named db_helpers
]


# Load Data

In [67]:
# connect to db
db_name = 'dota2_draft'
with open(os.path.expanduser('~/.pgpass')) as f:
    for line in f:
        host, port, db, user, password = [x.strip() for x in line.split(':')]
        if db == db_name:
            con, meta = connect(user=user, password=password, db=db, host=host, port=port)
            break
            
dota2_api, meta = connect(user, password, db, host, port)

In [61]:
query = '''
SELECT * FROM matches
LIMIT 5000;
'''
df = pd.read_sql(query, dota2_api)

In [62]:
df.head()

,match_id,duration,match_seq_num,players,radiant_win,start_time
0,3180507665,1969,2778110720,"[{u'backpack_2': 0, u'gold_spent': 22525, u'go...",True,1494832183
1,3180472977,3746,2778110721,"[{u'backpack_2': 0, u'gold_spent': 28795, u'go...",True,1494830922
2,3180507491,2554,2778110722,"[{u'backpack_2': 0, u'gold_spent': 11615, u'go...",False,1494832167
3,3180507549,2590,2778110723,"[{u'backpack_2': 0, u'gold_spent': 14415, u'go...",False,1494832166
4,3180511878,2434,2778110730,"[{u'backpack_2': 0, u'gold_spent': 19500, u'go...",True,1494832326


In [69]:
# grab heroes data
with open(os.path.expanduser('~/.pgpass')) as f:
    for line in f:
        host, port, db, user, password = [x.strip() for x in line.split(':')]
        if db == 'dota-draft':
            con, meta = connect(user=user, password=password, db=db, host=host, port=port)
            break
opendota, meta = connect(user, password, db, host, port)
heroes_df = pd.read_sql('''
SELECT * FROM heroes;
''', opendota)

In [70]:
heroes_df.head()

,id,attack_type,legs,localized_name,name,primary_attr,roles
0,44,Melee,2,Phantom Assassin,npc_dota_hero_phantom_assassin,agi,"{Carry,Escape}"
1,49,Melee,2,Dragon Knight,npc_dota_hero_dragon_knight,str,"{Carry,Pusher,Durable,Disabler,Initiator,Nuker}"
2,54,Melee,2,Lifestealer,npc_dota_hero_life_stealer,str,"{Carry,Durable,Jungler,Escape,Disabler}"
3,55,Melee,2,Dark Seer,npc_dota_hero_dark_seer,int,"{Initiator,Jungler,Escape,Disabler}"
4,58,Ranged,4,Enchantress,npc_dota_hero_enchantress,int,"{Support,Jungler,Pusher,Durable,Disabler}"


In [72]:
heroes_df.id.max()

114

# Preprocessing

In [105]:
players_list = df.iloc[0, 3]
radiant_heroes, dire_heroes = preprocessing.parse_players(players_list)
radiant_heroes, dire_heroes

(0    10
 1    14
 2    74
 3     5
 4    57
 Name: hero_id, dtype: int64, 5    106
 6     37
 7     65
 8    109
 9     32
 Name: hero_id, dtype: int64)

In [114]:
pd.get_dummies(radiant_heroes).reindex(columns=range(0, 115), fill_value=0).sum()

0      0
1      0
2      0
3      0
4      0
5      1
6      0
7      0
8      0
9      0
10     1
11     0
12     0
13     0
14     1
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
85     0
86     0
87     0
88     0
89     0
90     0
91     0
92     0
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
111    0
112    0
113    0
114    0
Length: 115, dtype: int64

In [118]:
%timeit preprocessing.make_dummy(radiant_heroes, dire_heroes)

100 loops, best of 3: 2.69 ms per loop


In [121]:
%timeit dummies = preprocessing.make_dummies(df)

KeyboardInterrupt: 

In [89]:
dummies.shape

(5000, 230)

In [93]:
dummies[(dummies['radiant_0'] == 1) | (dummies['dire_0'] == 1)].shape[0]

4

In [98]:
heroes = df['players'].map(preprocessing.parse_players)

In [122]:
%timeit     dummies_dfs = map(lambda x: preprocessing.make_dummy(*x), heroes)


1 loop, best of 3: 8.95 s per loop
